# https://prob.space/topics/sylk-Post1493775608cf71e92217 にいろいろ追加

In [1]:
!pwd
import sys
%reload_ext autoreload
%autoreload 2
%matplotlib inline
sys.executable

C:\Users\yokoi.shingo\my_notebook\py37\ProbSpace_game_comp\notebook


'C:\\Users\\yokoi.shingo\\Anaconda3\\envs\\py37\\python.exe'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score as acc
from sklearn.preprocessing import MultiLabelBinarizer
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

# データの読込
train = pd.read_csv("../orig/train_data.csv")
test = pd.read_csv('../orig/test_data.csv')

## feature_eng

In [3]:
# ヒーロー武器をベースの武器に変更
change_weapon = {
    "heroblaster_replica":  "hotblaster",
    "herobrush_replica": "hokusai", 
    "herocharger_replica":  "splatcharger",
    "heromaneuver_replica":"maneuver",
    "heroroller_replica":"splatroller",
    "heroshelter_replica":"parashelter",
    "heroshooter_replica":"sshooter",
    "heroslosher_replica":"bucketslosher",
    "herospinner_replica":"splatspinner",
    "octoshooter_replica":"sshooter"
                }

train = train.replace(change_weapon)
test = test.replace(change_weapon)

mlb = MultiLabelBinarizer()
mlb.fit([set(train['A4-weapon'].fillna("none").unique())])

MultiLabelBinarizer()

In [4]:
#mlb.classes_

In [5]:
train.head(3)

,id,period,game-ver,lobby-mode,lobby,mode,stage,A1-weapon,A1-rank,A1-level,A2-weapon,A2-rank,A2-level,A3-weapon,A3-rank,A3-level,A4-weapon,A4-rank,A4-level,B1-weapon,B1-rank,B1-level,B2-weapon,B2-rank,B2-level,B3-weapon,B3-rank,B3-level,B4-weapon,B4-rank,B4-level,y
0,1,2019-10-15T20:00:00+00:00,5.0.1,regular,standard,nawabari,sumeshi,sshooter_becchu,NaN,139,soytuber_custom,NaN,118.0,pablo_hue,NaN,13.0,hokusai,NaN,10.0,bold_7,NaN,28,hokusai_becchu,NaN,26.0,splatcharger,NaN,68.0,sharp_neo,NaN,31.0,1
1,2,2019-12-14T04:00:00+00:00,5.0.1,regular,standard,nawabari,arowana,parashelter_sorella,NaN,198,jetsweeper,NaN,77.0,campingshelter_camo,NaN,198.0,nzap85,NaN,123.0,momiji,NaN,83,squiclean_b,NaN,118.0,campingshelter,NaN,168.0,sputtery_clear,NaN,151.0,0
2,3,2019-12-25T14:00:00+00:00,5.0.1,gachi,standard,hoko,ama,nzap89,a-,114,quadhopper_black,a,68.0,prime_becchu,a,225.0,jetsweeper,a,107.0,bold_7,a-,50,nzap85,a+,163.0,prime_becchu,a-,160.0,dualsweeper_custom,a,126.0,0


In [6]:
test.head(3)

,id,period,game-ver,lobby-mode,lobby,mode,stage,A1-weapon,A1-rank,A1-level,A2-weapon,A2-rank,A2-level,A3-weapon,A3-rank,A3-level,A4-weapon,A4-rank,A4-level,B1-weapon,B1-rank,B1-level,B2-weapon,B2-rank,B2-level,B3-weapon,B3-rank,B3-level,B4-weapon,B4-rank,B4-level
0,1,2019-12-17T12:00:00+00:00,5.0.1,gachi,standard,area,hakofugu,prime_collabo,x,174,splatspinner,x,130.0,nzap89,x,127.0,nova_becchu,x,233.0,furo,x,160,nautilus47,x,151.0,l3reelgun_d,x,213.0,nzap89,x,306.0
1,2,2019-11-25T16:00:00+00:00,5.0.1,gachi,standard,asari,anchovy,prime_becchu,s+,363,nzap89,s+,59.0,dynamo_becchu,s+,36.0,rapid_becchu,s+,225.0,furo,s+,326,dualsweeper_custom,s+,289.0,prime_becchu,s+,147.0,splatroller,s+,156.0
2,3,2019-10-22T08:00:00+00:00,5.0.1,gachi,standard,area,mutsugoro,furo_deco,s,116,bold,s,125.0,nzap85,s,124.0,wakaba,s,181.0,splatroller,s,76,momiji,s,232.0,nzap83,s,183.0,promodeler_pg,s,105.0


In [7]:
print(len(train), len(test))

66125 28340


In [8]:
train.y.sum()

34696

In [9]:
# 追加

def add_num_rank(df):
    """
    rankは順位があるみたいなので順位数にする
    https://wiki.denfaminicogamer.jp/Splatoon2/%E3%82%A6%E3%83%87%E3%83%9E%E3%82%A8
    ウデマエとは、ガチマッチでの、自分の腕前を表す「レート」のようなもの
    ウデマエの段階。ウデマエは、一番低い「C-」からスタート
    C-→C→C+→B-→B→B+→A-→A→A+→S→S+→X
    S+0→S+1→S+2→S+3→→→S+9 2018年4月下旬のアップデートによって、S+10以上はウデマエXとなりました
    """
    dic_rank = {
        "x": 12,
        "s+": 11,
        "s": 10,
        "a+": 9,
        "a": 8,
        "a-": 7,
        "b+": 6,
        "b": 5,
        "b-": 4,
        "c+": 3,
        "c": 2,
        "c-": 1,
    }
    for s in ["A", "B"]:
        for i in [1, 2, 3, 4]:
            if df[f"{s}{i}-rank"].dtype.name in ['object', 'category']:
                df[f"{s}{i}-rank"] = df[f"{s}{i}-rank"].replace(dic_rank)
                df[f"{s}{i}-rank"] = df[f"{s}{i}-rank"].astype(float)
    return df


def get_seq_labels(seq, threshold=0):
    """
    seq : 時系列順のリスト
    threshold : level up のための最低試合数
    
    [3,3,3,4,4,4,4,4,7,7,7,7,2,2,2,1,1,1,2,8,8,8] : level
     => [1,1,1,1,1,1,1,1,2,2,2,2,3,3,3,4,4,4,3,2,2,2] : player id
    というように、レベルに応じてA1の特定を考えます
    
    時間の昇順に並び替えてるから、
    同じレベルなら同一playerとみなし、
    第二引数のthresholdの回数繰り返されて、レベルがインクりされてれば、同一playerと考えてるってこと
    """

    box = np.zeros(len(seq), dtype=int)  # 最終的にラベルが入るボックス
    count = 1  # label

    for _ in range(1000):
        # level : 時系列順のレベルでユニークなもの
        # s     : levelの値を格納
        
        ind = np.where(box == 0)[0][0]
        s = seq[ind]

        renew_box = []
        for i in range(len(seq)):
            if box[i] == 0:
                if s == seq[i]:
                    box[i] = count
                    renew_box.append(seq[i])
                elif (s + 1 == seq[i]) and (
                    (np.array(renew_box) == s).sum() >= threshold
                ):
                    s += 1
                    box[i] = count
            else:
                continue
        count += 1
        if (box == 0).sum() == 0:
            # box が全部埋まれば break
            break
    return box

In [10]:
def make_feature(df_train, df_test):
    train_num = len(df_train)
    #print(f"train_num: {train_num}")
    
    df = pd.concat([df_train, df_test]).reset_index(drop=True)
    
    # カテゴリ列を数値に変換
    #print(len(df))
    cat_cols = ["lobby-mode", "mode", "stage"]
    for c in cat_cols:
        vv, obj = pd.factorize(df[c])
        df[c] = vv
    #print(len(df))
    
    # その行のA,Bの武器存在したか1/0で表す（カウント数ではないので1/0のどちらかしか値とらない）
    A1 = ['A1-weapon', 'A2-weapon', 'A3-weapon', 'A4-weapon']
    B1 = ['B1-weapon', 'B2-weapon', 'B3-weapon', 'B4-weapon']
    t = mlb.transform(df[A1].fillna("none")[A1].values)
    t2 = mlb.transform(df[B1].fillna("none")[B1].values)
    for i in range(t.shape[1]):
        df["A-" + mlb.classes_[i]] = t[:, i]
        df["B-" + mlb.classes_[i]] = t2[:, i]
        
    # -------------------------------------------
    # 追記分
    
    # 参加人数列を追加
    df["A_cnt"] = df[["A-" + i for i in mlb.classes_]].sum(axis=1)
    df["B_cnt"] = df[["B-" + i for i in mlb.classes_]].sum(axis=1)
    
    # rankを数値に変換
    df = add_num_rank(df)
    
    # A1-player列作成
    sorted_df = df.sort_values(["period", "A1-level"]).reset_index()
    levels = sorted_df["A1-level"].tolist() # A1 level を時系列順にソートしたリスト
    a1_player = get_seq_labels(levels, threshold=20)  # 最低threshold回試合したらレベルアップしたと考えている
    sorted_df["A1-player"] = a1_player
    df = sorted_df
    
#    # A1-playerごとの勝率
#    df_new = None
#    for p_id in tqdm(np.unique(a1_player)):
#        df_player = df[df["A1-player"] == p_id]
#        for i, _df in df_player.groupby("A1-player"):
#            _df['A1-player_win_rate'] = _df["y"].sum() / len(_df["y"].dropna())  # 各A1プレイヤーごとの勝率
#            if df_new is None:
#                df_new = _df
#            else:
#                df_new = df_new.append(_df)
#    df = df_new
    
#    # A1-playerの武器ごとの勝率
#    df_new = None
#    for p_id in tqdm(np.unique(a1_player)):
#        df_player = df[df["A1-player"] == p_id]
#        for i, _df in df_player.groupby("A1-weapon"):
#            _df['A1-player_win_weapon_rate'] = _df["y"].sum() / len(_df["y"].dropna())  # 各A1プレイヤーの武器ごとの勝率
#            if df_new is None:
#                df_new = _df
#            else:
#                df_new = df_new.append(_df)
#    df = df_new
    # -------------------------------------------
        
    # レベルをクリッピング
    for i in ["A", "B"]:
        for j in ["1", "2", "3", "4"]:
            df[i+j+"-level"] = df[i+j+"-level"]//10  # レベルを1/10倍する
            df[i+j+"-level"] = df[i+j+"-level"].clip(0,30)  # レベルの下限=0, 上限=30にする
        
    #return df[:train_num], df[train_num:]
    
    df = df.sort_values(by='index')  # train→testの順番に直す
    #display(df)
    return df.iloc[:train_num].reset_index(drop=True), df.iloc[train_num:].reset_index(drop=True)

# 確認
tr, te = make_feature(train, test)
print(tr.shape)
print(te.shape)

(66125, 296)
(28340, 296)


In [11]:
#tr.to_csv("tr_feat_eng.csv", index=False)
#te.to_csv("te_feat_eng.csv", index=False)

## fit

In [12]:
#tr = pd.read_csv("tr_feat_eng.csv")
#te = pd.read_csv("te_feat_eng.csv")

#tr = pd.read_csv("tr_feat_eng_A1-player_win_weapon_rate.csv")
#te = pd.read_csv("te_feat_eng_A1-player_win_weapon_rate.csv")

In [13]:
#display(tr.head(2))

In [14]:
#display(te.head(2))

In [15]:
feature_cols = [
     'lobby-mode', 'mode', 'stage',
]
feature_cols.extend(["A-" + i for i in mlb.classes_])
feature_cols.extend(["B-" + i for i in mlb.classes_])

In [16]:
# 追加

feature_cols.extend([
    "A1-level", "A2-level", "A3-level", "A4-level", "B1-level", "B2-level", "B3-level", "B4-level", 
    "A1-rank", #"A2-rank", "A3-rank", "A4-rank", "B1-rank", "B2-rank", "B3-rank", "B4-rank",
    "A_cnt", "B_cnt",
    #"A1-player", 
    #"A1-player_win_rate", 
    #"A1-player_win_weapon_rate",
])


In [17]:
feature_cols

['lobby-mode',
 'mode',
 'stage',
 'A-52gal',
 'A-52gal_becchu',
 'A-52gal_deco',
 'A-96gal',
 'A-96gal_deco',
 'A-bamboo14mk1',
 'A-bamboo14mk2',
 'A-bamboo14mk3',
 'A-barrelspinner',
 'A-barrelspinner_deco',
 'A-barrelspinner_remix',
 'A-bold',
 'A-bold_7',
 'A-bold_neo',
 'A-bottlegeyser',
 'A-bottlegeyser_foil',
 'A-bucketslosher',
 'A-bucketslosher_deco',
 'A-bucketslosher_soda',
 'A-campingshelter',
 'A-campingshelter_camo',
 'A-campingshelter_sorella',
 'A-carbon',
 'A-carbon_deco',
 'A-clashblaster',
 'A-clashblaster_neo',
 'A-dualsweeper',
 'A-dualsweeper_custom',
 'A-dynamo',
 'A-dynamo_becchu',
 'A-dynamo_tesla',
 'A-explosher',
 'A-explosher_custom',
 'A-furo',
 'A-furo_deco',
 'A-h3reelgun',
 'A-h3reelgun_cherry',
 'A-h3reelgun_d',
 'A-hissen',
 'A-hissen_hue',
 'A-hokusai',
 'A-hokusai_becchu',
 'A-hokusai_hue',
 'A-hotblaster',
 'A-hotblaster_custom',
 'A-hydra',
 'A-hydra_custom',
 'A-jetsweeper',
 'A-jetsweeper_custom',
 'A-kelvin525',
 'A-kelvin525_becchu',
 'A-kelvin

In [18]:
# def cv_score(clf,X,y,n_splits=5,kf=StratifiedKFold(n_splits=5))->float:
kf = KFold(n_splits=5, random_state=0)
score = 0
counter = 1
#for train_index, valid_index in kf.split(tr, tr.y):
for train_index, valid_index in kf.split(train, train.y):
    train_X, valid_X = train.loc[train_index, :].copy(), train.loc[valid_index, :].copy()
    train_y, valid_y = train.y.iloc[train_index], train.y.iloc[valid_index]

    train_X, valid_X = make_feature(train_X, valid_X)
    
    clf = lgb.LGBMClassifier()
    clf.fit(train_X[feature_cols], train_y)

    preds = clf.predict(valid_X[feature_cols])
    acc_score = acc(valid_y, preds)
    print(f"fold{counter} score is :{acc_score}")
    score += acc_score
    counter += 1
print("average : ", round(score / 5, 5))

fold1 score is :0.547296786389414
fold2 score is :0.5469943289224952
fold3 score is :0.5482797731568998
fold4 score is :0.5440453686200378
fold5 score is :0.5431379962192817
average :  0.54595


## predict

In [19]:
# 提出用　全データで再学習してsubmitファイル作成
#tr, te = make_feature(train, test)
clf = lgb.LGBMClassifier().fit(tr[feature_cols], tr["y"])
pred_test = clf.predict(te[feature_cols])
pd.DataFrame({"id": range(len(pred_test)), "y": pred_test}).to_csv(
    "submission_sample_code_20201009.csv", index=False
)

In [20]:
# importanceを表示する
importance = pd.DataFrame(clf.booster_.feature_importance(importance_type='gain'), index=feature_cols, columns = ["f"])
importance.sort_values("f", ascending = False).head(10)

,f
B4-level,1836.142762
B1-level,1541.689179
B2-level,1521.799963
B3-level,1495.820762
A4-level,1432.622204
A2-level,1379.956471
A1-rank,1276.090108
A1-level,1266.443718
A3-level,1090.899340
stage,1053.871583


# 結果

average :  0.54072 # base

average :  0.55194 # A1-level 追加（クリッピングなし）+ A-cnt, B-cnt 追加 + A1-4-level, B1-4-level, A1-rank 追加 + A1-player_win_rate 追加 # submit score=0.548624 リークしてる

average :  0.59017 # A1-level 追加（クリッピングなし）+ A-cnt, B-cnt 追加 + A1-4-level, B1-4-level, A1-rank 追加 + A1-player_win_weapon_rate 追加 # submit score=0.531404 リークしてる

average :  0.54530 # A1-level 追加（クリッピングなし）+ A-cnt, B-cnt 追加 + A1-4-level, B1-4-level, A1-rank 追加

average :  0.54377 # A1-level 追加（クリッピングなし）+ A-cnt, B-cnt 追加 + A1-4-level, B1-4-level, A1-rank 追加 + A1-player 追加

average :  0.54323 # A1-level 追加（クリッピングなし）+ A-cnt, B-cnt 追加

average :  0.54361 # A1-level 追加（クリッピングなし）+ A-cnt, B-cnt 追加 + A1-rank 追加

average :  0.54361 # A1-level 追加（クリッピングなし）+ A-cnt, B-cnt 追加 + A1-level, A1-rank 追加

average :  0.54361 # A1-level 追加（クリッピングなし）+ A-cnt, B-cnt 追加 + A1-level, B1-level, A1-rank, A2-rank 追加

average :  0.54335 # A1-level 追加（クリッピングなし）+ A-cnt, B-cnt 追加 + A1-4-level, B1-4-level, A1-4-rank, B1-4-rank 追加

average :  0.54179 # A1-level 追加（クリッピングなし）+ A-cnt, B-cnt 追加 + A1-player 追加

<br>

# old（make_feature(train_X, valid_X) もした時の結果）

average :  0.5407 # base

average :  0.54126 # hero武器の置換なし

average :  0.54234 # hero武器の置換なし + A1-level 追加（クリッピングなし）

average :  0.54231 # hero武器の置換なし + A1-level 追加（クリッピングあり） # submit score=0.544954

average :  0.5414 # hero武器の置換なし + A1-level 追加（クリッピングあり） + B1-level 追加

average :  0.54081 # hero武器の置換なし + A1-level 追加（クリッピングあり） + A1-rank 追加

average :  0.53893 # hero武器の置換なし + A1-level 追加（クリッピングあり） + A1-rank 追加 + A2-rank 追加

average :  0.53949 # hero武器の置換なし + A1-level 追加（クリッピングあり） + A-cnt 追加

average :  0.52103 # hero武器の置換なし + A1-level 追加（クリッピングなし） + A1-player 追加